In [256]:
import requests
import warnings
from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import unittest
import tables
import pickle
import pyarrow.parquet as pq
import pyarrow as pa
import plotly.io as pio
import math
import investpy
import sklearn.preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import subprocess
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [119]:
pio.templates.default = "plotly_dark"

In [242]:
oi = yf.Ticker("TSLA")

In [259]:
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True)

ERROR: This script is intended to run on Linux only.


CompletedProcess(args='curl -fsSL https://ollama.com/install.sh | sh', returncode=1)